In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense
from keras.models import Sequential
from keras.layers import Activation
from tensorflow.keras.layers import Dropout
from matplotlib import pyplot as plt

In [2]:

# Set up directories
parent_dir = r"C:\Users\HP\Desktop\Resume Projects\Python Projects\ISL TPU Project\data"
output_dir = os.path.join(parent_dir, "processed")
image_path = r"C:\Users\HP\Desktop\Resume Projects\Python Projects\ISL TPU Project"
# Load and preprocess the data
data = []
labels = []
labels_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,
                   'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,
                   'Z':25}
num_classes = 26

# Iterate over each character folder
for char in "ABCDEFGHIJKLMNOPQRSTUVWXYZ":
    char_folder = os.path.join(output_dir, char)

    # Process each image for the character
    for i in range(1200):
        image_path = os.path.join(char_folder, f"{char}_{i}_processed.jpg")
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Assuming images are grayscale
        image = cv2.resize(image, (64, 64))  # Resize images to a consistent size
        data.append(image)
        labels.append(labels_dict[char]) 
data = np.array(data)
# Normalize pixel values to the range [0, 1]
data = data.astype("float32") / 255.0
labels = keras.utils.to_categorical(labels)   




KeyboardInterrupt: 

In [3]:
(train_data, test_data, train_labels, test_labels) = train_test_split(
    data, labels, test_size=0.2, random_state=42
)

In [7]:
# Build the CNN model
model = keras.Sequential()
model.add(Conv2D(64, kernel_size=4, strides=1, activation='relu', input_shape=(64, 64, 1)))
model.add(Conv2D(64, kernel_size=4, strides=2, activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(128, kernel_size=4, strides=1, activation='relu'))
model.add(Conv2D(128, kernel_size=4, strides=2, activation='relu'))
model.add(Dropout(0.5))
model.add(Conv2D(256, kernel_size=4, strides=1, activation='relu'))
model.add(Conv2D(256, kernel_size=4, strides=2, activation='relu'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dense(26, activation='softmax'))
model.summary()   



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 61, 61, 64)        1088      
                                                                 
 conv2d_13 (Conv2D)          (None, 29, 29, 64)        65600     
                                                                 
 dropout_6 (Dropout)         (None, 29, 29, 64)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 26, 26, 128)       131200    
                                                                 
 conv2d_15 (Conv2D)          (None, 12, 12, 128)       262272    
                                                                 
 dropout_7 (Dropout)         (None, 12, 12, 128)       0         
                                                                 
 conv2d_16 (Conv2D)          (None, 9, 9, 256)        

In [8]:

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss',patience=2)

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=["accuracy"])

model_hist= model.fit(train_data, train_labels, epochs=1,batch_size=32,verbose=2)

score = model.evaluate(x=test_data, y=test_labels,verbose=0)
print('Test accuracy:', score[1])




780/780 - 297s - loss: 0.2015 - accuracy: 0.9429 - 297s/epoch - 381ms/step
Test accuracy: 0.9985576868057251


In [9]:
loss, accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy*100:.2f}%")

195/195 [==============================] - 17s 89ms/step - loss: 0.0047 - accuracy: 0.9986
Test Loss: 0.0047
Test Accuracy: 99.86%


In [23]:
model.save("sign_recognition_model.h5")